# EO Products as Output Data

An Application that creates EO product files that need to be staged-out must also create in the output directory a STAC Catalog that enumerates and documents the produced files.

The STAC Catalog created by the Application must include metadata elements defining properties as the start time & end time or the geographical footprint. These two elements are the minimum set of metadata elements to enable their discovery but depending on the context, the application can convey further elements.

It is assumed that all output files not referenced in the STAC Catalog local file are not relevant to the process and can be discarded by any subsequent action and thus not staged out.

## The EO application

The EO application takes a Sentinel-2 scene and detects water bodies using the NDWI index and the Ostu threshold

In [3]:
import os
import click
import pystac
import rasterio
from skimage.filters import threshold_otsu
from rasterio.mask import mask
from pyproj import Transformer
from shapely import box
from loguru import logger
import rasterio
import pystac
import shutil
import rio_stac
import numpy as np

np.seterr(divide="ignore", invalid="ignore")


def crop(asset: pystac.Asset, bbox, epsg):
    """_summary_

    Args:
        asset (_type_): _description_
        bbox (_type_): _description_
        epsg (_type_): _description_

    Returns:
        _type_: _description_
    """
    with rasterio.open(asset.get_absolute_href()) as src:

        transformer = Transformer.from_crs(epsg, src.crs, always_xy=True)

        minx, miny = transformer.transform(bbox[0], bbox[1])
        maxx, maxy = transformer.transform(bbox[2], bbox[3])

        transformed_bbox = box(minx, miny, maxx, maxy)

        logger.info(f"Crop {asset.get_absolute_href()}")

        out_image, out_transform = rasterio.mask.mask(
            src, [transformed_bbox], crop=True
        )
        out_meta = src.meta.copy()

        out_meta.update(
            {
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
            }
        )

        return out_image.astype(np.float32), out_meta


def threshold(data):
    """Returns the Otsu threshold of a numpy array"""
    return data > threshold_otsu(data[np.isfinite(data)])


def normalized_difference(array1, array2):
    """Returns the normalized difference of two numpy arrays"""
    return (array1 - array2) / (array1 + array2)


def aoi2box(aoi):
    """Converts an area of interest expressed as a bounding box to a list of floats"""
    return [float(c) for c in aoi.split(",")]


def get_asset(item, common_name):
    """Returns the asset of a STAC Item defined with its common band name"""
    for _, asset in item.get_assets().items():
        if not "data" in asset.to_dict()["roles"]:
            continue

        eo_asset = pystac.extensions.eo.AssetEOExtension(asset)
        if not eo_asset.bands:
            continue
        for b in eo_asset.bands:
            if (
                "common_name" in b.properties.keys()
                and b.properties["common_name"] == common_name
            ):
                return asset


@click.command(
    short_help="Crop",
    help="Water bodies detection using the Normalized Difference Water Index (NDWI) and Otsu thresholding.",
)
@click.option(
    "--input-item",
    "item_url",
    help="STAC Item URL or staged STAC catalog",
    required=True,
)
@click.option(
    "--aoi",
    "aoi",
    help="Area of interest expressed as a bounding box",
    required=True,
)
@click.option(
    "--epsg",
    "epsg",
    help="EPSG code",
    required=True,
)
@click.option(
    "--band",
    "bands",
    help="Common band name",
    required=True,
    multiple=True,
)
def main(item_url, aoi, bands, epsg):

    if os.path.isdir(item_url):
        catalog = pystac.read_file(os.path.join(item_url, "catalog.json"))
        item = next(catalog.get_items())
    else:
        item = pystac.read_file(item_url)

    logger.info(f"Read {item.id} from {item.get_self_href()}")

    cropped_assets = {}

    for band in bands:
        asset = get_asset(item, band)
        logger.info(f"Read asset {band} from {asset.get_absolute_href()}")

        if not asset:
            msg = f"Common band name {band} not found in the assets"
            logger.error(msg)
            raise ValueError(msg)

        bbox = aoi2box(aoi)

        out_image, out_meta = crop(asset, bbox, epsg)

        cropped_assets[band] = out_image[0]

    nd = normalized_difference(cropped_assets[bands[0]], cropped_assets[bands[1]])

    water_bodies = threshold(nd)

    out_meta.update(
        {
            "dtype": "uint8",
            "driver": "COG",
            "tiled": True,
            "compress": "lzw",
            "blockxsize": 256,
            "blockysize": 256,
        }
    )

    water_body = "otsu.tif"

    with rasterio.open(water_body, "w", **out_meta) as dst_dataset:
        logger.info(f"Write otsu.tif")
        dst_dataset.write(water_bodies, indexes=1)

    logger.info(f"Creating a STAC Catalog")
    cat = pystac.Catalog(id="catalog", description="water-bodies")

    if os.path.isdir(item_url):
        catalog = pystac.read_file(os.path.join(item_url, "catalog.json"))
        item = next(catalog.get_items())
    else:
        item = pystac.read_file(item_url)

    os.makedirs(item.id, exist_ok=True)
    shutil.copy(water_body, item.id)

    out_item = rio_stac.stac.create_stac_item(
        source=water_body,
        input_datetime=item.datetime,
        id=item.id,
        asset_roles=["data", "visual"],
        asset_href=os.path.basename(water_body),
        asset_name="data",
        with_proj=True,
        with_raster=False,
    )

    cat.add_items([out_item])

    cat.normalize_and_save(
        root_href="./", catalog_type=pystac.CatalogType.SELF_CONTAINED
    )

    os.remove(water_body)

    logger.info("Done!")




## Run the application

Print the application help

In [4]:
from click.testing import CliRunner

runner = CliRunner()
result = runner.invoke(main, ['--help'])

print(result.output)

Usage: main [OPTIONS]

  Water bodies detection using the Normalized Difference Water Index (NDWI) and
  Otsu thresholding.

Options:
  --input-item TEXT  STAC Item URL or staged STAC catalog  [required]
  --aoi TEXT         Area of interest expressed as a bounding box  [required]
  --epsg TEXT        EPSG code  [required]
  --band TEXT        Common band name  [required]
  --help             Show this message and exit.



Invoke the application

In [5]:
arguments = ["--input-item", "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
                "--aoi", "-121.399,39.834,-120.74,40.472",
                "--epsg", "EPSG:4326",
                "--band", "green",
                "--band", "nir"]


In [6]:
runner = CliRunner()
result = runner.invoke(main, args=arguments)

print(result.output)

2024-04-04 07:41:01.600 | INFO     | __main__:main:126 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-04-04 07:41:02.231 | INFO     | __main__:main:132 - Read asset green from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-04 07:41:04.933 | INFO     | __main__:crop:39 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-04-04 07:41:49.638 | INFO     | __main__:main:132 - Read asset nir from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-04-04 07:41:51.296 | INFO     | __main__:crop:39 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-04-04 07:42:32.511 | INFO     | __main__:main:1

## Inspect the application results

In [8]:
cat = pystac.read_file("catalog.json")

cat.describe()

* <Catalog id=catalog>
  * <Item id=S2B_10TFK_20210713_0_L2A>


In [9]:
item = next(cat.get_items())

for key, asset in item.get_assets().items():
    print(key, asset.href, asset.media_type, asset.roles)

data ./otsu.tif image/tiff; application=geotiff ['data', 'visual']


In [11]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'S2B_10TFK_20210713_0_L2A',
 'properties': {'proj:epsg': 32610,
  'proj:geometry': {'type': 'Polygon',
   'coordinates': [[[636990.0, 4410550.0],
     [691590.0, 4410550.0],
     [691590.0, 4482600.0],
     [636990.0, 4482600.0],
     [636990.0, 4410550.0]]]},
  'proj:bbox': [636990.0, 4410550.0, 691590.0, 4482600.0],
  'proj:shape': [7205, 5460],
  'proj:transform': [10.0,
   0.0,
   636990.0,
   0.0,
   -10.0,
   4482600.0,
   0.0,
   0.0,
   1.0],
  'proj:projjson': {'$schema': 'https://proj.org/schemas/v0.4/projjson.schema.json',
   'type': 'ProjectedCRS',
   'name': 'WGS 84 / UTM zone 10N',
   'base_crs': {'name': 'WGS 84',
    'datum': {'type': 'GeodeticReferenceFrame',
     'name': 'World Geodetic System 1984',
     'ellipsoid': {'name': 'WGS 84',
      'semi_major_axis': 6378137,
      'inverse_flattening': 298.257223563}},
    'coordinate_system': {'subtype': 'ellipsoidal',
     'axis': [{'name': 'Geodetic latitude',
       